In [2]:
import os
from ECG.ecg import read_ecg
from ECG.ecg_sig import band_pass_filter
from ECG.ecg import make_df_summary
# file  =  os.path.join(os.path.dirname(os.path.realpath(__file__)),"a01.dat")
# filer  =  os.path.join(os.path.dirname(os.path.realpath(__file__)),"a01r.dat")

file  =  "a01.dat"
filer  =  "a01r.dat"

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split,GridSearchCV

In [3]:
data1 = pd.read_csv("freq_DT_Ampli_freqa1.csv")
data2 = pd.read_csv("freq_DT_Ampli_freqa2.csv")
data3 = pd.read_csv("freq_DT_Ampli_freqa3.csv")
data4 = pd.read_csv("freq_DT_Ampli_freqa4.csv")
data = pd.concat([data1,data2,data3,data4])
data = data.drop("Unnamed: 0",1)

In [4]:
data.head(50)

,Freq,DT,Amplitude,Freqr
0,0.100100,0.892857,294.476190,0.20020
1,0.450450,0.866957,305.652174,0.15015
2,0.450450,0.900435,301.739130,0.10010
3,0.100100,0.849565,315.217391,0.30030
4,0.450450,0.820400,304.520000,0.15015
5,0.250250,0.849130,298.304348,0.30030
6,0.350350,0.879565,306.956522,0.15015
7,0.200200,0.767308,293.807692,0.15015
8,0.150150,0.798800,305.080000,0.30030
9,0.100100,0.713571,286.178571,0.35035


In [5]:
data["Amplitude"] = data["Amplitude"]/data["Amplitude"].max()
data

,Freq,DT,Amplitude,Freqr
0,0.100100,0.892857,0.166889,0.20020
1,0.450450,0.866957,0.173222,0.15015
2,0.450450,0.900435,0.171005,0.10010
3,0.100100,0.849565,0.178643,0.30030
4,0.450450,0.820400,0.172581,0.15015
...,...,...,...,...
105,0.150150,0.882174,0.278437,0.30030
106,0.200200,0.747037,0.262375,0.35035
107,0.100100,0.840833,0.274888,0.30030
108,0.500501,0.875000,0.288697,0.35035


In [6]:
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

In [7]:
y = data["Freqr"].values
X = data.drop(['Freqr',"Amplitude"],1).values
# X = data.drop(['Freqr'],1).values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42)


In [25]:
%%time
# benchmark
benchmark = np.sum(np.abs(y_test - X_test[:,0]))/len(y_test)
benchmark

CPU times: user 62 µs, sys: 0 ns, total: 62 µs
Wall time: 66 µs


0.1362634310611189

In [16]:
%%time
#SVM
clf = SVR(gamma='scale', C=1.0, epsilon=0.1,kernel='rbf')
clf.fit(X_train, y_train) 
clf.predict(X_test)
mean_absolute_error(y_test,clf.predict(X_test))

CPU times: user 47.3 ms, sys: 2.68 ms, total: 50 ms
Wall time: 48.9 ms


0.08360284464692334

In [15]:
%%time
#ElasticNet
regr = ElasticNet(random_state=0,alpha=0.5,l1_ratio=0.5)
regr.fit(X_train, y_train) 
regr.predict(X_test)
mean_absolute_error(y_test,regr.predict(X_test))

CPU times: user 1.81 ms, sys: 626 µs, total: 2.43 ms
Wall time: 1.9 ms


0.08310700430198058

In [18]:
%%time
# Random forest
fest = RandomForestRegressor(max_depth=3, random_state=0,
                             n_estimators=100,criterion='mse')
fest.fit(X_train, y_train)
fest.predict(X_test)
mean_absolute_error(y_test,fest.predict(X_test))

CPU times: user 129 ms, sys: 2.91 ms, total: 132 ms
Wall time: 130 ms


0.08197942674561245

0.1362634310611189